In [1]:
using Pkg
pkg"activate ."
pkg"instantiate"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


I am given a vector `v` of length `d` and wish to permute it according to some permutation `p`.

In [2]:
d = 10
@show d
v = rand(10)
@show v
@show typeof(v)
import Random
p = Random.randperm(d)
@show p
@show typeof(p);

d = 10
v = [0.19897, 0.143348, 0.145023, 0.0891942, 0.572756, 0.537881, 0.264896, 0.956121, 0.909801, 0.386333]
typeof(v) = Array{Float64,1}
p = [3, 2, 7, 6, 9, 4, 1, 5, 10, 8]
typeof(p) = Array{Int64,1}


Doing this permutation is easy with Julia's indexing:

In [3]:
v[p]

10-element Array{Float64,1}:
 0.14502310546324804
 0.14334832301613387
 0.2648959007333165 
 0.5378812297108322 
 0.9098005835242389 
 0.0891942162577819 
 0.19896973691987618
 0.5727559441660186 
 0.3863326895524921 
 0.9561208200424756 

We can turn this into a function, which Julia will compile a method for based on the types of `v` and `p`.

In [4]:
unstatic_sort(v, p) = v[p]

@time unstatic_sort(v, p)
@show unstatic_sort(v, p);

  0.027094 seconds (5.10 k allocations: 221.314 KiB)
unstatic_sort(v, p) = [0.145023, 0.143348, 0.264896, 0.537881, 0.909801, 0.0891942, 0.19897, 0.572756, 0.386333, 0.956121]


The `@time` macro shows us the time it took to run that function the first time, which includes compilation time. If want to properly benchmark it, we should use `BenchmarkTools.jl`.

In [5]:
using BenchmarkTools
@btime unstatic_sort($v, $p);

  76.139 ns (2 allocations: 176 bytes)


But what if we want to sort many vectors with the same permutation? We can encode the permutation in the type domain and compile a specialized function to do this much faster. To make such a [value type](https://docs.julialang.org/en/v1/base/base/#Base.Val), our permutation must have `isbits(p) == true`.

In [6]:
isbits(p)

false

To remedy this, we could convert `p` to a tuple which has a static length, unlike a vector (which it currently is). However, then we won't be able to do the indexing trick of `v[p]` to permute the entries of the vector. Instead, we'll convert it to a `SVector` or static vector from `StaticArrays.jl`. This supports indexing like vectors, but is backed by a tuple of fixed length.

In [7]:
using StaticArrays
p_static = SVector{d}(p)

10-element SArray{Tuple{10},Int64,1,10}:
  3
  2
  7
  6
  9
  4
  1
  5
 10
  8

In [8]:
isbits(p_static)

true

We can then create a function `static_sort` which dispatches to a different method for every permutation. We do this by replacing the second argument with a parametric type `Val{p}`. We can then create a value type from our `isbits` vector `p_static` and act our new function on it.

In [9]:
function static_sort(v, ::Val{p}) where {p}
    v[p]
end

p_val = Val(p_static)

@time static_sort(v, p_val)

  0.246776 seconds (284.86 k allocations: 14.774 MiB, 2.51% gc time)


10-element SArray{Tuple{10},Float64,1,10}:
 0.14502310546324804
 0.14334832301613387
 0.2648959007333165 
 0.5378812297108322 
 0.9098005835242389 
 0.0891942162577819 
 0.19896973691987618
 0.5727559441660186 
 0.3863326895524921 
 0.9561208200424756 

We can see it's relatively slow to compile. However, let's benchmark the runtime.

In [10]:
@btime static_sort($v, $p_val);

  4.694 ns (0 allocations: 0 bytes)


Less than 5 nanoseconds! Much faster now. We can see why this is by inspecting the typed code:

In [11]:
@code_typed static_sort(v, p_val)

CodeInfo(
1 ─       (Base.arraysize)(v, 1)::Int64
│   %2  = (Base.arrayref)(true, v, 3)::Float64
│   %3  = (Base.arrayref)(true, v, 2)::Float64
│   %4  = (Base.arrayref)(true, v, 7)::Float64
│   %5  = (Base.arrayref)(true, v, 6)::Float64
│   %6  = (Base.arrayref)(true, v, 9)::Float64
│   %7  = (Base.arrayref)(true, v, 4)::Float64
│   %8  = (Base.arrayref)(true, v, 1)::Float64
│   %9  = (Base.arrayref)(true, v, 5)::Float64
│   %10 = (Base.arrayref)(true, v, 10)::Float64
│   %11 = (Base.arrayref)(true, v, 8)::Float64
│   %12 = (StaticArrays.tuple)(%2, %3, %4, %5, %6, %7, %8, %9, %10, %11)::NTuple{10,Float64}
│   %13 = %new(SArray{Tuple{10},Float64,1,10}, %12)::SArray{Tuple{10},Float64,1,10}
└──       return %13
) => SArray{Tuple{10},Float64,1,10}

Or even the native assembly instructions:

In [12]:
@code_native static_sort(v, p_val)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[9]:2 within `static_sort'
	pushl	%ebp
	decl	%eax
	movl	%esp, %ebp
; │┌ @ abstractarray.jl:927 within `getindex'
; ││┌ @ indexing.jl:208 within `_getindex' @ indexing.jl:213
; │││┌ @ indexing.jl:244 within `macro expansion'
; ││││┌ @ array.jl:729 within `getindex'
	incl	%ecx
	pushl	%esi
	pushl	%ebx
	decl	%eax
	movl	8(%esi), %eax
	decl	%eax
	cmpl	$2, %eax
	jbe	L140
	decl	%eax
	cmpl	$6, %eax
	jbe	L181
	decl	%eax
	cmpl	$8, %eax
	jbe	L222
	decl	%eax
	cmpl	$9, %eax
	je	L263
	decl	%eax
	movl	(%esi), %eax
	decl	%esp
	movl	16(%eax), %ecx
	decl	%esp
	movl	(%eax), %eax
	decl	%eax
	movl	8(%eax), %esi
	decl	%eax
	movl	48(%eax), %edx
	decl	%eax
	movl	40(%eax), %ecx
	decl	%esp
	movl	64(%eax), %edx
	decl	%esp
	movl	24(%eax), %ebx
	decl	%eax
	movl	32(%eax), %ebx
	decl	%esp
	movl	72(%eax), %esi
	decl	%eax
	movl	56(%eax), %eax
; │└└└└
	decl	%esp
	movl	%ecx, (%edi)
	decl	%eax
	movl	%esi, 8(%edi)
	decl	%eax
	movl	%edx, 16(%edi)
	decl	%eax
	movl	%ec

This can be compared to the dynamic version:

In [13]:
@code_typed unstatic_sort(v, p)

CodeInfo(
1 ──       (Base.arraysize)(v, 1)::Int64
└───       goto #18 if not true
2 ── %3  = (Core.tuple)(p)::Tuple{Array{Int64,1}}
│    %4  = (Base.arraysize)(v, 1)::Int64
│    %5  = (Base.slt_int)(%4, 0)::Bool
│    %6  = (Base.ifelse)(%5, 0, %4)::Int64
│    %7  = (Base.arraylen)(p)::Int64
│    %8  = (Base.sle_int)(0, %7)::Bool
│    %9  = (Base.bitcast)(UInt64, %7)::UInt64
│    %10 = (Base.ult_int)(0x0000000000000000, %9)::Bool
│    %11 = (Base.and_int)(%8, %10)::Bool
└───       goto #4 if not %11
3 ── %13 = (Base.arrayref)(false, p, 1)::Int64
└───       goto #5
4 ──       goto #5
5 ┄─ %16 = φ (#3 => false, #4 => true)::Bool
│    %17 = φ (#3 => %13)::Int64
│    %18 = φ (#3 => 2)::Int64
└───       goto #6
6 ── %20 = (Base.not_int)(%16)::Bool
└───       goto #12 if not %20
7 ┄─ %22 = φ (#6 => true, #11 => %28)::Bool
│    %23 = φ (#6 => %17, #11 => %41)::Int64
│    %24 = φ (#6 => %18, #11 => %42)::Int64
│    %25 = (Base.sle_int)(1, %23)::Bool
│    %26 = (Base.sle_int)(%23, %6)::Bool
│  

And its assembly instructions:

In [14]:
@code_native unstatic_sort(v, p)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[4]:1 within `unstatic_sort'
	incl	%ecx
	pushl	%edi
	incl	%ecx
	pushl	%esi
	incl	%ecx
	pushl	%ebp
	incl	%ecx
	pushl	%esp
	pushl	%ebx
	decl	%eax
	subl	$64, %esp
	vpxor	%xmm0, %xmm0, %xmm0
	vmovdqa	%xmm0, (%esp)
	decl	%eax
	movl	%esi, %ebx
	decl	%eax
	movl	$0, 16(%esp)
	decl	%eax
	movl	%ebx, 56(%esp)
	decl	%eax
	movl	$95475360, %eax         ## imm = 0x5B0D6A0
	addl	%eax, (%eax)
	addb	%al, (%eax)
	calll	*%eax
	decl	%ecx
	movl	%eax, %esi
	decl	%eax
	movl	$2, (%esp)
	decl	%ecx
	movl	(%esi), %eax
	decl	%eax
	movl	%eax, 8(%esp)
	decl	%eax
	movl	%esp, %eax
	decl	%ecx
	movl	%eax, (%esi)
	decl	%esp
	movl	(%ebx), %edi
	decl	%esp
	movl	8(%ebx), %esp
; │┌ @ abstractarray.jl:927 within `getindex'
; ││┌ @ multidimensional.jl:641 within `_getindex'
	decl	%eax
	movl	$95486944, %eax         ## imm = 0x5B103E0
	addl	%eax, (%eax)
	addb	%al, (%eax)
	movl	$1616, %esi             ## imm = 0x650
	movl	$16, %edx
	decl	%esp
	movl	%esi, %edi
	calll	*%eax


Note that if we wish to permute vectors by a different permutation `p'` instead of `p`, the function `static_sort` will compile a new method (which takes about a quarter-second on my laptop) for the new permutation. So this is only a good way to go if we wish to permute many vectors by the same permutation.